In [1]:
import os
import re
from glob import glob
from tqdm import tqdm
import pandas as pd

### Get class2ind dictionary

In [2]:
anno_base = '/home/longteng/ssd/ucf_anno'
class_ind_file = os.path.join(anno_base, 'classInd.txt')

with open(class_ind_file, 'r') as fp:
    lines = fp.readlines()
    lines = [line.rstrip() for line in lines]

    
# a dictionary {'applyeyemakers' : 1, ...}
class2ind = {line.split(' ')[1].lower() : line.split(' ')[0] for line in lines}

### Get path2clsind dictionary

In [3]:
video_base = '/home/longteng/ssd/ucf101/'

video_pattern = os.path.join(video_base, '*.avi')

video_paths = glob(video_pattern)

path2clsind = dict()

for video_path in video_paths:
    
    clsname = os.path.split(video_path)[-1].split('_')[1]
    clsind  = class2ind[clsname.lower()]
    
    path2clsind[video_path] = clsind

In [4]:
path2clsind_df = pd.DataFrame.from_dict({'path':path2clsind.keys(), 'label':path2clsind.values()}, orient = 'columns')
path2clsind_df.head(3)

,path,label
0,/home/longteng/ssd/ucf101/v_PlayingCello_g16_c...,59
1,/home/longteng/ssd/ucf101/v_Billiards_g19_c04.avi,12
2,/home/longteng/ssd/ucf101/v_Biking_g18_c04.avi,11


### Get csv for train_csv, each row is "path, label\n"

In [7]:
train1_file = os.path.join(anno_base, 'trainlist01.txt')
train2_file = os.path.join(anno_base, 'trainlist02.txt')
train3_file = os.path.join(anno_base, 'trainlist03.txt')

train_file = train1_file # Change this

with open(train1_file, 'r') as fp:
    lines = fp.readlines()
    lines = [line.rstrip() for line in lines]
    
train_csv = train_file.replace('.txt', '.csv')

with open(train_csv, 'w+') as fp:

    new_lines = []
    for line in lines:
        new_line = re.sub('[a-zA-Z0-9]*/', video_base, line)
        new_lines.append(new_line + '\n')
        
    fp.writelines(new_lines)

### Get csv for test_csv, each row is "path, label\n"

In [14]:
test1_file = os.path.join(anno_base, 'testlist01.txt')
test2_file = os.path.join(anno_base, 'testlist01.txt')
test3_file = os.path.join(anno_base, 'testlist01.txt')

test_file = test3_file
test_dict = dict()

with open(test_file, 'r') as fp:
    lines = fp.readlines()
    lines = [line.rstrip() for line in lines]
    
for line in tqdm(lines):
    v_fn = line.split('/')[1]
    v_path = os.path.join(video_base, v_fn)
    
    row_pos = path2clsind_df.path.str.contains(v_fn)
    label = path2clsind_df.loc[row_pos,'label']
    label = label.values[0]

    test_dict[v_path] = label

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3783/3783 [00:14<00:00, 252.35it/s]


In [15]:
test_path2clsind_df = pd.DataFrame.from_dict(test_dict, orient='index')
test_path2clsind_df.to_csv('test02.csv')
test_path2clsind_df.head(3)

,0
/home/longteng/ssd/ucf101/v_ApplyEyeMakeup_g01_c01.avi,1
/home/longteng/ssd/ucf101/v_ApplyEyeMakeup_g01_c02.avi,1
/home/longteng/ssd/ucf101/v_ApplyEyeMakeup_g01_c03.avi,1
